In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot
import seaborn as sns
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
import pickle
import os
import NN

import dataset_utils
import importlib
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import LinearLocator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Conv3D, MaxPooling3D, Embedding, LSTM, Bidirectional, Reshape, Concatenate

In [39]:
DATA_DIR = "sd_indvidual_3_class_data"
NUM_POINTS = 200
NUM_CLASSES = 3
BATCH_SIZE = 32
train_points, test_points, train_labels, test_labels, CLASS_MAP = dataset_utils.parse_dataset(NUM_POINTS, DATA_DIR)

processing class: jumping
processing class: speed_walking
processing class: walking


In [4]:
#data parsing

import os
import pickle

# Define the base directory
base_dir = os.getcwd()
base_dir = os.path.join(base_dir, 'sd_indvidual_3_class_data')

# Initialize lists to store processed train and test data with labels
train_data = []
train_labels = []
test_data = []
test_labels = []

# Iterate over each class
for class_name in os.listdir(base_dir):
    class_path = os.path.join(base_dir, class_name)
    
    # Ensure it's a directory
    if os.path.isdir(class_path):
        for split in ['train', 'test']:
            split_path = os.path.join(class_path, split)
            
            # Process files in the train/test folder
            for filename in os.listdir(split_path):
                file_path = os.path.join(split_path, filename)
                
                # Load the data (assuming it's a pickle file)
                with open(file_path, 'rb') as file:
                    data = pickle.load(file)
                
                # Limit to 100 slots if there are more than 100
                if len(data) > 100:
                    data = data[:100]
                
                # Store the processed data and corresponding label
                if split == 'train':
                    train_data.append(data)
                    train_labels.append(class_name)  # Assuming the class_name is the label
                elif split == 'test':
                    test_data.append(data)
                    test_labels.append(class_name)  # Assuming the class_name is the label

# Now you have train_data, train_labels, test_data, and test_labels

In [5]:
print(test_labels)

['jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'jumping', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking', 'speed_walking'

In [9]:
import numpy as np
# Assuming you want to see the shape of each array in train_data
for i, data in enumerate(train_data):
    print(f"Shape of train_data[{i}]: {np.array(data).shape}")

# Optionally, you can convert the entire list to a NumPy array if all arrays are of the same shape
train_data_array = np.array(train_data)
print(f"Shape of train_data_array: {train_data_array.shape}")

NameError: name 'np' is not defined

In [43]:
print(train_points.shape)
print(test_points.shape)
print(train_labels.shape)
print(test_labels.shape)

(480, 200)
(120, 200)
(480,)
(120,)


In [26]:
print(train_points[0])

[ 5.435e-01 -2.783e+00  1.450e+02  0.000e+00  1.648e-02  1.267e-01
  1.760e+02 -9.092e-01 -1.812e+00  7.085e-01  1.880e+02  5.435e-01
 -3.757e-01 -1.252e-01  8.486e-01 -1.318e+00  8.575e-02  1.880e+02
  1.847e-01  5.435e-01  1.750e+02  1.570e+02  7.168e-01 -1.252e-01
 -9.092e-01 -3.406e-01 -3.643e-01  6.431e-01  1.580e+02  7.168e-01
  1.343e-01  4.546e-01  2.496e-01 -2.742e+00  0.000e+00 -8.472e-01
  1.437e+00  1.399e-01  1.252e-01  0.000e+00  6.758e-01  1.648e-02
  1.460e+02  1.252e-01 -1.252e-01  0.000e+00  1.252e-01  4.111e-01
 -6.274e-01  0.000e+00 -2.012e-01 -6.187e-01  1.612e+00  7.827e-01
 -6.187e-01  1.510e+02 -1.476e+00  1.450e+02  2.496e-01  5.464e-01
  0.000e+00  1.660e+02  2.070e+02 -4.224e-02  8.486e-01  0.000e+00
  2.025e+00  7.515e-01  1.120e+02 -1.875e+00  1.437e+00  1.804e-01
  1.847e-01  7.168e-01  1.730e+02  0.000e+00  1.460e+02  1.648e-02
  5.573e-02  4.629e-01  1.399e-01  1.120e+02 -3.201e+00  1.490e+02
  1.750e+02  1.700e+02  1.252e-01  7.085e-01  6.616e-01  1.730

In [21]:
from tensorflow import data as tf_data
import keras
#import PointNET

keras.utils.set_random_seed(seed=42)

In [22]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization

# Define the input shape (5 features)
sequence_length = 100  # Example: Fixed sequence length of 100 timesteps

input_shape = (sequence_length, 5)

# Define the model
model = models.Sequential()

# 1. CNN layers
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
# model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu'))
# model.add(layers.MaxPooling1D(pool_size=2))

# 2. Flatten the output before feeding into Dense layers
model.add(layers.Flatten())

# 3. Fully connected layer
model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(3, activation='softmax'))


c:\Users\Yang Qian\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 98, 64)         │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 49, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       200,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 201,987 (789.01 KB)

 Trainable params: 201,987 (789.01 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
# training
history = model.fit(train_points, train_labels, epochs=30, validation_data=(test_points, test_labels), verbose=1, callbacks=[early_stopping])

Epoch 1/30


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(32, 100), dtype=float32). Expected shape (None, 100, 5), but input has incompatible shape (32, 100)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 100), dtype=float32)
  • training=True
  • mask=None

In [70]:
test_loss, test_acc = model.evaluate(test_points, test_labels, verbose=1)

6/6 [==============================] - 0s 6ms/step - loss: 1.6617 - accuracy: 0.3148
